<a href="https://colab.research.google.com/github/heidingaway/datapeople/blob/main/GCThesaurus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import os
import zipfile
import yaml
import re
import unicodedata

In [25]:
## Link to CSV

src = "https://canada.multites.net/cst/EAEAD1E6-7DD2-4997-BE7F-40BFB1CBE8A2/CST20240911.csv"

# Create the DataFrame from all the retrieved records
df = pd.read_csv(src)

# Add column names
df.columns = ['subject', 'predicate', 'object']

In [26]:
# Create the DataFrame from all the retrieved records
print(df.head())

                          subject predicate                object
0               2-spirited people       Use     Two-spirit people
1               2019-nCoV disease       Use  Coronavirus diseases
2  2019 novel coronavirus disease       Use  Coronavirus diseases
3                 2SLGBTQ+ people       Use      2SLGBTQI+ people
4                2SLGBTQI+ people    French    Personne 2ELGBTQI+


In [27]:
# Define the strings you want to add
prefix = "[["
suffix = "]]"

# Use the apply method with a lambda function to modify the column content
df['linkedsubject'] = df['subject'].apply(lambda x: f"{prefix}{x}{suffix}")

# Define the headings you want to add
heading2 = "## "

# Use the apply method with a lambda function to modify the column content
df['h2'] = df['predicate'].apply(lambda x: f"{heading2}{x}")

# identify title
df = df.rename(columns={'object': 'title'}).sort_values(by='title', ascending=False)

# Display the modified DataFrame
print(df.head())

               subject predicate               title        linkedsubject  \
1475         Art works    French         Œuvre d'art        [[Art works]]   
10161     Human beings    French         Être humain     [[Human beings]]   
19251    Sports events    French   Événement sportif    [[Sports events]]   
4635   Cultural events    French  Événement culturel  [[Cultural events]]   
19712    Tax avoidance    French    Évitement fiscal    [[Tax avoidance]]   

              h2  
1475   ## French  
10161  ## French  
19251  ## French  
4635   ## French  
19712  ## French  


In [28]:
cleandf = df.drop(['subject','predicate'], axis=1)


In [29]:
print(cleandf.head())

                    title        linkedsubject         h2
1475          Œuvre d'art        [[Art works]]  ## French
10161         Être humain     [[Human beings]]  ## French
19251   Événement sportif    [[Sports events]]  ## French
4635   Événement culturel  [[Cultural events]]  ## French
19712    Évitement fiscal    [[Tax avoidance]]  ## French


In [ ]:
n

In [45]:
# Group by 'title' and 'h2' and aggregate 'linkedsubject' with comma separation
grouped_df = cleandf.groupby(['title', 'h2'])['linkedsubject'].apply(lambda x: ', '.join(x)).reset_index()

# Pivot the grouped DataFrame
pivoted_df = grouped_df.pivot(index='title', columns='h2', values='linkedsubject').reset_index()

print(pivoted_df)

h2                                                title ## Broader Term  \
0     "Bicycle paths" replaces "Cycling trails" as p...             NaN   
1     "Biochemicals" replaces "Biochemical products"...             NaN   
2     "Coasts" replaces "Coastlands" as preferred te...             NaN   
3     "Cultural groups" replaces "Cultural minoritie...             NaN   
4     "Demining" replaces "Mine clearing" as preferr...             NaN   
...                                                 ...             ...   
7583                                   Évitement fiscal             NaN   
7584                                 Événement culturel             NaN   
7585                                  Événement sportif             NaN   
7586                                        Être humain             NaN   
7587                                        Œuvre d'art             NaN   

h2              ## French                               ## History note  \
0                     Na

In [46]:
import hashlib

output_dir = 'grouped_csvs4'  # Replace with your desired directory path

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Set the maximum file name length (adjust as needed)
max_file_name_length = 150

# Iterate through each group (title) and export to CSV
for title, group_data in pivoted_df.groupby('title'):
    # Check if the title is too long
    if len(title) > max_file_name_length:
        # Shorten the file name using hashing
        file_hash = hashlib.sha256(title.encode()).hexdigest()
        file_path = os.path.join(output_dir, f'{file_hash}.csv')
    else:
        # Use the original title for the file name, replacing invalid characters
        # This replaces '/' with '_' to avoid creating unintended directories
        file_name = title.replace('/', '_')
        file_path = os.path.join(output_dir, f'{file_name}.csv')

    group_data.to_csv(file_path, index=False)  # index=False to avoid saving index

print(f"CSV files exported to: {output_dir}")

CSV files exported to: grouped_csvs4
